# Import

In [20]:
from sklearn.preprocessing import OneHotEncoder
from decisiontree import *

# Data Preprocessing

In [13]:
## Read csvs
train_df = pd.read_csv('data/email_classificatioin/train.csv', index_col=0)
test_df = pd.read_csv('data/email_classificatioin/test.csv', index_col=0)

In [14]:
## Filtering column "mail_type"
train_x = train_df[['mail_type']]
train_x = train_x.fillna(value='None')
train_y = train_df[['label']]

test_x = test_df[['mail_type']]
test_x = test_x.fillna(value='None')

## Do one hot encoding of categorical feature
feat_enc = OneHotEncoder()
feat_enc.fit(np.vstack([train_x, test_x]))
train_x_featurized = feat_enc.transform(train_x)
test_x_featurized = feat_enc.transform(test_x)

c:\Anaconda\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Anaconda\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


# Classification Task

In [22]:
tree = decision_tree_algorithm(train_df, ml_task="classification", max_depth=3)

TypeError: '<' not supported between instances of 'float' and 'str'